### Used to convert pydantic schema definitions for data types into json objects used by react to generate the interface for forms

### !!! Only works inside monorepo and not a containerized api instance !!!

In [1]:
import sys
sys.path.append('..')
!{sys.executable} -m pip install jsonref

from schemas import collection_schema
from schemas import dataset_schema
from schemas import datafeed_schema
from schemas import anomaly_schema
from schemas import detector_schema
from schemas import prediction_schema
from schemas import health_schema

from pathlib import Path
import json
import jsonref
import datetime
import copy

In [2]:
options={}
options["Collection"]=collection_schema.Collection_DB
options["Datafeed"]=datafeed_schema.Datafeed_DB
options["Dataset"]=dataset_schema.Dataset_DB
options["Anomaly"]=anomaly_schema.Anomaly_DB
options["Detector"]=detector_schema.Detector_DB
options["Prediction"]=prediction_schema.Prediction_DB
options["Health"]=health_schema.Health_DB

PATH_TO_REACT_PROJECT="../../apps/time-series/src/"

In [3]:
#dataset=jsonref.replace_refs(dataset_schema.Dataset_DB.model_json_schema())
#dataset_payload=jsonref.replace_refs(dataset_schema.Dataset_Base.model_json_schema())
#print(json.dumps(dataset["properties"], indent=2))
#print(json.dumps(dataset["properties"]["health"], indent=2))

In [4]:
def convert_types(item):
    """ Converts pydantic type/format into a generic type"""

    if "anyOf" in item: #when using optional value (value | None = None)
        if item["anyOf"][0]["format"] =="uuid": #should be Id
            return "id"
        else: #otherwise return object
            return "object" 

    if "allOf" in item: 
        if "enum" in item["allOf"][0]: #select using Enum values
            return "select"
        else:
            return "object" #is a db object
    
    if (item["type"]=="array"):
        if item["items"]["type"] == "string": #list array
            return "list"
        if item["items"]["type"] == "object": #list of database children
            return "children"

    if (item["type"]=="string"):
        if "format" not in item:
            return item["type"]
        if item["format"]=="date-time":
            return "datetime"
        if item["format"]=="uuid":
            return "id"
        
    elif (item["type"]=="integer" or item["type"]=="float" ):
            return "number"

    else:
        return item["type"]
    

def get_title(item):
    #Select Enums have title inside the Enum class
    if "allOf" in item:
        return item["allOf"][0]["title"] 
    else:
        return item["title"]
    

def define_filtering(item):
    #db objects will not be filtered, otherwise get generic empty value and rule based on type
    if item["type"] in ["id", "object", "children"]:
        item["filter"]=False
    else:
        item["filter"]=True
        item["filter_empty"] = defineEmptyFilter(item)
        item["filter_rule"] = defineFilterRule(item)

def defineEmptyFilter(item):       
    if item["type"] in ["string","number","boolean"]:
        return ""
    if item["type"] in ['select','list']:
        return []
    if item["type"] in ["datetime"]:
        return {"timespan_begin":"","timespan_end":""}

def defineFilterRule(item):    
    if item["type"] == "string":
        return "equals"
    if item["type"] == "number":
        return ">"
    if item["type"] in ['select','list']:
        return "any"

def defineFormProperties(schema_attr,item):
    #db objects can not be edited, other values get a default to populate forms

    if item["source"]=='db':
        item["editable"]=False

    else:
        item["editable"]=True

    if "default" in schema_attr:
        item["default"]=schema_attr['default']

def add_pydantic_data(schema_attr,model_entry):
    """ Adds the source, type, title and optional "options" for the schema entry """

    if "source" in schema_attr:
        model_entry["source"]=schema_attr["source"]
    else:
        model_entry["source"]="main"

    model_entry["type"]=convert_types(schema_attr)
    model_entry["title"]=get_title(schema_attr)

    #if type is select or list, get the options
    if model_entry["type"] =='select':
        options=[]
        for enum_item in schema_attr["allOf"][0]["enum"]:
            options.append({"value":enum_item, "label":enum_item})
        model_entry["options"]=options

    if model_entry["type"] =='list':
        model_entry["options"]=[]


def make_modelData_array(properties):
    model_array=[]
    for attr_name in properties:
        schema_attr=properties[attr_name] #the pydantic schema data for the key 'attr_name'

        model_data={}
        model_data["name"]=attr_name

        add_pydantic_data(schema_attr,model_data)
        defineFormProperties(schema_attr,item=model_data)
        define_filtering(model_data)
        model_array.append(model_data)
    return model_array

def make_modelData_object(properties):
    model_data={}
    for attr_name in properties:
        schema_attr=properties[attr_name] #the pydantic schema data for the key 'attr_name'

        model_data[attr_name]={}
        model_data[attr_name]["name"]=attr_name

        add_pydantic_data(schema_attr,model_data[attr_name])
        defineFormProperties(schema_attr,item=model_data[attr_name])
        define_filtering(model_data[attr_name])

    return model_data

In [5]:
def load_pydantic_model(SELECTION):
    model_schema=jsonref.replace_refs(options[SELECTION].model_json_schema()) #resolves references to enums
    properties=model_schema["properties"]
    return properties


def load_json_file(model):
    path=Path(PATH_TO_REACT_PROJECT) / "metadata" / str(model+"ModelData.json")
    f = open(path)
    json_data=json.load(f)
    return json_data

def save_json_file(model,json_data):
    path=Path(PATH_TO_REACT_PROJECT) / "metadata" / str(model+"ModelData.json")
    with open(path,"w") as json_file:
        json.dump(json_data,json_file)

In [6]:
#SELECTION="Dataset"
#SELECTION="Datafeed"
#SELECTION="Anomaly"
#SELECTION="Detector"
#SELECTION="Prediction"

data_models=["Collection","Datafeed","Dataset","Anomaly","Detector","Prediction","Health"]
#data_models=["Anomaly"]
for model in data_models:
    print(model)
    properties=load_pydantic_model(model)

    json_data=load_json_file(model)
    json_data["model_attributes"]=make_modelData_array(properties)


    save_json_file(model,json_data)

Collection
Datafeed
Dataset
Anomaly
Detector
Prediction
Health
